# Linear Regression using La Classy

In [31]:
import { parse } from "jsr:@std/csv@0.218";
import {
  ClassificationReport,
  Matrix,
  useSplit,
  CategoricalEncoder,
  rmse,
} from "jsr:@lala/appraisal@0.7.5";
import {
  GradientDescentSolver,
  mse,
  adamOptimizer
} from "jsr:@lala/classy@1.2.2";


We first load our dataset `iris.csv`.

In [32]:
const data = parse(Deno.readTextFileSync("../datasets/winequality-red.csv"));

Skip the first row (header).

In [33]:
data.shift()

[
  "fixed acidity",
  "volatile acidity",
  "citric acid",
  "residual sugar",
  "chlorides",
  "free sulfur dioxide",
  "total sulfur dioxide",
  "density",
  "pH",
  "sulphates",
  "alcohol",
  "quality"
]

We can now get the predictor and target variables from the dataset.

In [34]:
const x = data.map((fl, i) => [...fl.slice(0, 8), ...fl.slice(9, 11)]);

const X = new Matrix(x, "f64");
X.slice(0, 10);


0,7.4,0.7,0,1.9,0.076,11,34,0.9978,0.56,9.4
1,7.8,0.88,0,2.6,0.098,25,67,0.9968,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15,54,0.997,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17,60,0.998,0.58,9.8
4,7.4,0.7,0,1.9,0.076,11,34,0.9978,0.56,9.4
5,7.4,0.66,0,1.8,0.075,13,40,0.9978,0.56,9.4
6,7.9,0.6,0.06,1.6,0.069,15,59,0.9964,0.46,9.4
7,7.3,0.65,0,1.2,0.065,15,21,0.9946,0.47,10
8,7.8,0.58,0.02,2,0.073,9,18,0.9968,0.57,9.5
9,7.5,0.5,0.36,6.1,0.071,17,102,0.9978,0.8,10.5


Let's use pH value as our target variable.

In [35]:
const y = new Matrix(data.map((fl) => [fl[8]]), "f64");
y.slice(0, 10);


0,3.51
1,3.2
2,3.26
3,3.16
4,3.51
5,3.51
6,3.3
7,3.39
8,3.36
9,3.35


In [36]:
[X.shape, y.shape]

[ [ 1599, 10 ], [ 1599, 1 ] ]

We now split our dataset for training and testing purposes. 

In [37]:
const [[x_train, y_train], [x_test, y_test]] = useSplit(
  { ratio: [7, 3], shuffle: true },
  X,
  y
);
x_train.slice(0, 10)

0,7.4,0.7,0,1.9,0.076,11,34,0.9978,0.56,9.4
1,7.8,0.76,0.04,2.3,0.092,15,54,0.997,0.65,9.8
2,7.4,0.66,0,1.8,0.075,13,40,0.9978,0.56,9.4
3,7.3,0.65,0,1.2,0.065,15,21,0.9946,0.47,10
4,7.8,0.58,0.02,2,0.073,9,18,0.9968,0.57,9.5
5,7.5,0.5,0.36,6.1,0.071,17,102,0.9978,0.8,10.5
6,6.7,0.58,0.08,1.8,0.097,15,65,0.9959,0.54,9.2
7,5.6,0.615,0,1.6,0.089,16,59,0.9943,0.52,9.9
8,8.9,0.62,0.18,3.8,0.176,52,145,0.9986,0.88,9.2
9,8.9,0.62,0.19,3.9,0.17,51,148,0.9986,0.93,9.2


Now that we have prepared our inputs, we can initialize our solver. Since we are performing linear regression, let's try out an Ordinary Least Squares solver.

In [38]:
const solver = new GradientDescentSolver({
    loss: mse(),
    optimizer: adamOptimizer(11, 1)
});

We can then train our model using the data we acquired.

In [39]:
solver.train(x_train, y_train, {
  fit_intercept: true,
  learning_rate: 0.01,
  epochs: 500
});

The model is trained, now it is time to evaluate its performance on our testing dataset

In [40]:
const res = solver.predict(x_test)
res.shape

[ 480, 1 ]

In [41]:
[res.row(0), y_test.row(0)]

[ Float64Array(1) [ 3.360432435136356 ], Float64Array(1) [ 3.2 ] ]

Let's calculate the RMSE.

In [42]:
rmse(y_test.data, res.data)

0.11203936034057838